<a href="https://colab.research.google.com/github/Son0208/prac_class/blob/main/final/Bert5_%EC%98%81%ED%99%94%EB%A6%AC%EB%B7%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

http://yonghee.io/bert_binary_classification_naver/

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.3MB 4.2MB/s 
     |████████████████████████████████| 3.3MB 35.0MB/s 
     |████████████████████████████████| 901kB 38.4MB/s 


In [ ]:
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [ ]:
import os

n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
  print(torch.cuda.get_device_name(i))

1
Tesla K80


In [ ]:
!git clone https://github.com/Son0208/repository.git

fatal: destination path 'repository' already exists and is not an empty directory.


In [ ]:
train = pd.read_csv('repository/training.csv')
test = pd.read_csv('repository/validation.csv')

In [ ]:
# LABEL 컬럼에 [0,1,0,...,0] 데이터가 되도록 도전
# train.Labels.str.join(" ")[0]
# train.Labels.str.join("|").str.get_dummies()


In [ ]:
train = train[['ID','Tweet']].join(train.Labels.str.join("|").str.get_dummies())
train.rename(columns = {' ' : '10'}, inplace = True)  # train.columns = ['ID', 'Tweet', '10', '0','1','2','3','4','5','6','7','8','9']
train = train[['ID','Tweet','0','1','2','3','4','5','6','7','8','9','10']]

print(train.shape)
print(test.shape)

In [ ]:
train

,ID,Tweet,0,1,2,3,4,5,6,7,8,9,10
0,1,NO JOKE I WILL HOP ON A PLANE RN! (Well after ...,1,1,0,0,0,0,0,0,0,0,1
1,2,BanMediaHouse whose is responsible for spreadi...,0,0,0,0,0,0,1,0,0,0,0
2,3,Im waiting for someone to say to me that all t...,0,0,0,1,1,0,0,0,0,0,1
3,4,He is a liar. Proven day night. Time again. Li...,0,0,0,0,0,0,1,0,0,0,0
4,5,"NEW: U.S. CoronaVirus death toll reaches 4,000...",0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,Life Insurance? I wonder if policies are payin...,0,0,0,0,1,1,0,1,0,0,1
4996,4997,My cousin passed away from the corona virus to...,0,0,0,0,1,1,0,0,0,0,1
4997,4998,I guess Mother Nature really hates us. Yellows...,1,1,0,1,0,1,0,0,0,1,1
4998,4999,So question for the day that isnt related to c...,1,1,0,0,0,0,0,0,0,1,1


In [ ]:
nscs

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [ ]:
Tweet_bert = ["[CLS] " + str(s) + " [SEP]" for s in train.Tweet]
Tweet_bert[:5]

['[CLS] NO JOKE I WILL HOP ON A PLANE RN! (Well after COVID19 lol) [SEP]',
 '[CLS] BanMediaHouse whose is responsible for spreading Fake and communal Stories in this pandemic Corona situation. [SEP]',
 '[CLS] Im waiting for someone to say to me that all this corona thing is just an April fools joke [SEP]',
 '[CLS] He is a liar. Proven day night. Time again. Lies when the truth will do. COVID19 [SEP]',
 '[CLS] NEW: U.S. CoronaVirus death toll reaches 4,000 after nearly 900 new deaths were reported today (BNO News) Covid19 CoronaVirusOutbreak [SEP]']

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(s) for s in Tweet_bert]
print(tokenized_texts[0])

['[CLS]', 'NO', 'J', '##OK', '##E', 'I', 'W', '##IL', '##L', 'H', '##OP', 'ON', 'A', 'PL', '##AN', '##E', 'RN', '!', '(', 'Well', 'after', 'CO', '##VI', '##D', '##19', 'lo', '##l', ')', '[SEP]']


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [ ]:
MAX_LEN = 128
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')
input_ids[0]

In [ ]:
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
    
print(attention_masks[0])

In [ ]:
train[].values

In [ ]:
# b = pd.DataFrame(train.Labels.str.split(' '))
# b = pd.DataFrame(train.Labels.str.split(' ').tolist())
b = pd.DataFrame(train.Labels.str.split(' ').tolist(), index=train.ID).stack()
b = b.reset_index()[['ID',0]]
b.columns = ['ID','Labels']
c = train['Labels'].tolist()
train['Labels'].values

In [ ]:
train_inputs, validation_inputs, train_labels, validation_labels = \
train_test_split(input_ids, train['Labels'].values, random_state=42, test_size=0.1)

train_masks, validation_masks, _, _ = \
train_test_split(attention_masks,input_ids,random_state=42,test_size=0.1)

In [ ]:
input_ids[0]

In [ ]:
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

TypeError: ignored

In [ ]:
BATCH_SIZE = 2

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=BATCH_SIZE)

In [ ]:
sentences = test['document']
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
labels = test['label'].values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

In [ ]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

In [ ]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝
total_steps = len(train_dataloader) * epochs

# lr 조금씩 감소시키는 스케줄러
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시 함수
def format_time(elapsed):
    # 반올림
    elapsed_rounded = int(round((elapsed)))
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch   500  of  67,500.    Elapsed: 0:00:58.
  Batch 1,000  of  67,500.    Elapsed: 0:01:57.
  Batch 1,500  of  67,500.    Elapsed: 0:02:56.
  Batch 2,000  of  67,500.    Elapsed: 0:03:55.
  Batch 2,500  of  67,500.    Elapsed: 0:04:54.
  Batch 3,000  of  67,500.    Elapsed: 0:05:53.
  Batch 3,500  of  67,500.    Elapsed: 0:06:52.
  Batch 4,000  of  67,500.    Elapsed: 0:07:51.
  Batch 4,500  of  67,500.    Elapsed: 0:08:50.
  Batch 5,000  of  67,500.    Elapsed: 0:09:49.
  Batch 5,500  of  67,500.    Elapsed: 0:10:48.
  Batch 6,000  of  67,500.    Elapsed: 0:11:46.
  Batch 6,500  of  67,500.    Elapsed: 0:12:44.
  Batch 7,000  of  67,500.    Elapsed: 0:13:42.
  Batch 7,500  of  67,500.    Elapsed: 0:14:41.
  Batch 8,000  of  67,500.    Elapsed: 0:15:39.


KeyboardInterrupt: ignored

In [ ]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

NameError: ignored